# Семинар №2. Префиксное дерево и шинглы

**План**
1. Про лабы
2. Пишем руками руками префиксное дерево
2. Пишем алгоритм шинглов
3. Посмотрим на библиотеки, которые сами все это умеют

In [1]:
!pip install pymorphy3

In [2]:
from pymorphy3 import MorphAnalyzer
from nltk.tokenize import word_tokenize
from string import punctuation
from functools import cache
from typing import Tuple
import hashlib
from pprint import pprint

import nltk
nltk.download('punkt')

morph = MorphAnalyzer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Прочитаем и подготовим данные

In [3]:
with open('text_sem2.txt', encoding='utf-8') as f:
    text = f.read()

In [4]:
@cache
def process_word(word):
    if (word not in punctuation) and (word.isalpha()):
        word = morph.parse(word.lower())[0].normal_form
        return word

In [5]:
def preprocess_text(text):
    words = word_tokenize(text)
    res = []
    for word in words:
        word = process_word(word)
        if word is not None:
            res.append(word)
    return res

In [6]:
words_lst = preprocess_text(text)
words_lst_unique_sorted = [el for el in sorted(set(words_lst))]
words_lst_unique_sorted[:10]

['origanum',
 'vulgare',
 'а',
 'аврор',
 'аврора',
 'аврорат',
 'аврората',
 'автор',
 'агония',
 'агрессивный']

In [7]:
len(words_lst_unique_sorted)

2792

## Строим префиксное дерево

In [8]:
class PrefixTree:
    def __init__(self):
        self.vocab = {}

    def __add(self, word, vocab) -> dict:
        if len(word) > 0:
            if word[0] not in vocab:
                vocab[word[0]] = {'[END]': False}
            vocab[word[0]] = self.__add(word[1:], vocab=vocab[word[0]])
        else:
            vocab['[END]'] = True
        return vocab

    def add(self, word) -> None:
        self.vocab = self.__add(word, self.vocab)

    def __find(self, word, vocab) -> bool:
        if len(word) > 0:
            if word[0] not in vocab:
                return False
            return self.__find(word[1:], vocab=vocab[word[0]])
        return vocab.get('[END]', False)

    def find(self, word) -> bool:
        return self.__find(word, self.vocab)

    def remove(self, word) -> None:
        pass

In [9]:
tree = PrefixTree()
tree.add('волшебник')
tree.add('волк')
tree.add('')
pprint(tree.vocab)
tree.find('')
# tree.remove('волшебник')

{'[END]': True,
 'в': {'[END]': False,
       'о': {'[END]': False,
             'л': {'[END]': False,
                   'к': {'[END]': True},
                   'ш': {'[END]': False,
                         'е': {'[END]': False,
                               'б': {'[END]': False,
                                     'н': {'[END]': False,
                                           'и': {'[END]': False,
                                                 'к': {'[END]': True}}}}}}}}}}


True

## Алгоритм шинглов

Данные алгоритм был придуман, чтобы искать плагиат в текстах. Сейчас существуют более совершенные с точки зрения качества, но менее быстрые алгориитмы, однако шинглированеие все еще используется.

Алгоритм следующий:
- Нормализуем текст (лемматизация, удаление стоп-слов и пунктуации)
- Разбиваем его на шинглы: окна в N текстов с отступом в K текстов (N > K)
- Считаем хэши по шинглам
- Сравниваем контрльные суммы. Если находим много совпадений, то считаем, что тексты очень похожи.

In [10]:
texts = [
    'Кошки являются одними из самых популярных домашних животных. Они обладают прекрасным инстинктом охотника и неповторимым характером. Кошки известны своей независимостью, но при этом могут быть очень преданными и ласковыми к своим хозяевам. Они мастера по комфортному расслаблению и настоящим ценителям уютного места для сна. Кошки также отличаются своей элегантностью и грациозностью. Их изящные движения и манера двигаться делают их настоящими королевами, даже когда они просто играют или прогуливаются по дому.',
    'Кошки - удивительные создания с мягкой шерстью и умными глазами. Они способны привносить радость и умиротворение в нашу жизнь. Кошки обладают способностью самостоятельно следить за собой, что делает их идеальными домашними животными для тех, кто ценит чистоту и порядок. Они часто проявляют свою ласку и привязанность к своим хозяевам. Игры с кошками могут стать незабываемым развлечением, их ловкость и быстрота поражают наблюдателей. Кошки - это прекрасные компаньоны, которые могут дарить нам уют и радость каждый день.',
    'Кошки - это загадочные и красивые создания, что позволяет им занимать особое место в наших сердцах. Их глаза отражают таинственность и мудрость, а их мягкая шерсть манит гладить их без конца. Кошки могут быть самостоятельными, но при этом они жаждут внимания и ласки от своих хозяев. Они обладают уникальным характером, который можно описать как гордый и независимый, но в то же время ласковый и игривый. Наблюдать, как кошка мягко протягивает лапу или усаживается на свое любимое место, отдает нам чувство покоя и спокойствия. Кошки - прекрасные спутники, которые делают нашу жизнь более интересной и яркой.'
    'Кошки — загадочные и прекрасные существа, которые завораживают своей грацией и неповторимым характером. Они способны сделать нашу жизнь более яркой и насыщенной. Кошки известны своей независимостью, но несмотря на это, они могут быть верными и преданными друзьями. Они обладают уникальной способностью принимать наши эмоции и поддерживать нас в трудные моменты. Кошки также являются отличными охотниками и отлично подходят для тех, кто хочет иметь домашнего питомца, не требующего особого ухода.',
    'Кошки — истинные королевы нашего дома. Они обладают особой грацией и достоинством, которые притягивают взгляды и вызывают восхищение. Кошки являются мастерами по созданию уюта и комфорта в своем окружении. Они обожают ласку и внимание со стороны своих хозяев и готовы доставить им радость и умиротворение своим присутствием. Кошки способны понять нас без слов и стать настоящими спутниками в наших ежедневных заботах.',
    'Кошки — таинственные создания, полные загадок и загадочности. Они притягивают наше внимание своими умными и интригующими глазами. Кошки обладают тонким чувством эстетики и красоты, которое отражается в их грациозных движениях и элегантности. Они могут быть игривыми и требовать нашего внимания, но в то же время они умеют наслаждаться тишиной и уединением. Кошки — это идеальные компаньоны для тех, кто ценит спокойствие и гармонию в своей жизни.'
]

In [11]:
class Shingler:
    def __init__(self,):
        pass

    def fit(self, data):
        # hashlib.md5(t.encode()).hexdigest()
        pass

    def get_closest(self, text, n=3):
        pass

In [12]:
shgl = Shingler()
shgl.fit(texts)

In [13]:
shgl.get_closest(texts[2], n=3)

## Библиотеки

**Префиксное дерево**

[Дока](https://pygtrie.readthedocs.io/en/latest/)

In [14]:
!pip install pygtrie -q

In [15]:
import pygtrie

In [16]:
t = pygtrie.CharTrie()
t['wombat'] = True
t.has_subtrie('wo'), t.has_key('wo'), t.has_key('wombat')

(True, False, True)

Давайте положим сюда текст, с которым работали выше

In [17]:
# code
...

**Шинглы** (char-based)

[Дока](https://github.com/ulf1/kshingle)

In [18]:
!pip install "kshingle>=0.10.0,<1" -q

  Preparing metadata (setup.py) ... done


In [19]:
import kshingle as ks

In [20]:
shingles = ks.shingleset_k("abc", k=3)
print(shingles)

shingles = ks.shingleset_range("abc", 2, 3)
print(shingles)

shingles = ks.shingleset_list("abc", [1, 3])
print(shingles)

{'ab', 'bc', 'c', 'a', 'b', 'abc'}
{'ab', 'bc', 'abc'}
{'c', 'a', 'abc', 'b'}


In [21]:
metric = ks.jaccard_strings("Bericht", "berichten", k=5)
metric

0.5128205128205128

Давайте сравним наши тексты из пункта про шинглы